Objectif : 

Recherche des doublons, explorer les lignes budgets liées et produire un csv lié (simple)

Outil : 
Echantillon de 20k fichiers 

In [4]:
import pandas as pd 

df_doc = pd.read_parquet('../data/doc_budgetaire_20.parquet')
df_budget = pd.read_parquet('../data/bloc_budget_20.parquet')

In [15]:
df_doc['IdEtab'] = df_doc['IdEtab'].astype(str)
df_doc['Siret'] = df_doc['Siret'].astype(str)

In [26]:
id_unique = df_doc.drop_duplicates(['Siret', 'IdEtab'], keep= False)
ligne_uniques = df_doc[~df_doc.index.isin(id_unique.index)]
ligne_uniques[['Id_Fichier', 'Siret', 'IdEtab']]

,Id_Fichier,Siret,IdEtab


C'est confirmé, sur l'échantillon IdEtab et Siret sont les mêmes (on sait jamais)

In [32]:
doc_doublon = df_doc[df_doc.duplicated(subset=['Siret', 'Exer', 'NatDec', 'NumDec','CodTypBud'], keep = False)]
doc_doublon[['Id_Fichier', 'Siret', 'NumDec', 'date']].sort_values(['Siret', 'NumDec']).head(10)

,Id_Fichier,Siret,NumDec,date
0,790376,20000480200027,1,2021-11-03 07:23:51.768000+00:00
0,791034,20000480200027,1,2021-11-08 15:01:39.443000+00:00
0,681458,20000797900012,None,2021-04-01 10:43:44.440000+00:00
0,681423,20000797900012,None,2021-04-01 10:43:44.440000+00:00
0,804914,20002715900017,None,2021-12-22 16:15:22+00:00
0,813538,20002715900017,None,2022-02-14 10:55:28+00:00
0,818103,20003419700018,None,2022-03-09 14:48:59.207000+00:00
0,818102,20003419700018,None,2022-03-09 14:55:03.125000+00:00
0,809763,20003510300015,12,2022-02-02 15:29:49+00:00
0,807281,20003510300015,12,2022-01-04 16:28:06+00:00


On va séparer en deux catégories : ceux qui ont la même date et ceux qui ont des dates différentes

In [42]:
doublon_meme_date = doc_doublon[doc_doublon.duplicated(subset= ['date'], keep = False)].sort_values(['Siret', 'date'])
doublon_diff_date = doc_doublon[~doc_doublon.duplicated(subset= ['date'], keep = False)].sort_values(['Siret', 'date'])

doublon_meme_date.to_csv('../data/explo/doc_budgetaire_doublon_meme_dates')
doublon_diff_date.to_csv('../data/explo/doc_budgetaire_doublon_diff_dates')
doc_doublon.to_csv('../data/explo/doc_budgetaire_doublon')
print('pure doublon',len(doublon_meme_date))
print('modif posteriori', len(doublon_diff_date))

pure doublon 141
modif posteriori 201


--------
Explo des lignes budgets mêmes dates 

-------------

In [43]:
doublon_meme_date.head(10)

,Id_Fichier,Nomenclature,Exer,Siret,Siren,CodColl,LibelleColl,DteStr,date,DteDec,...,DepFoncN2,RecFoncN2,DepInvN2,RecInvN2,CodTypBud,CodBud,ProjetBudget,Affect,SpecifBudget,FinJur
0,681458,M14-M14_COM_500_3500,2021,20000797900012,200007979,901,SIVOS DE BALLON-CIRE D'AUNIS,2021-04-01,2021-04-01 10:43:44.440000+00:00,2021-03-25,...,0.0,0.0,0.0,0.0,P,00,False,None,<NA>,<NA>
0,681423,M14-M14_COM_500_3500,2021,20000797900012,200007979,901,SIVOS DE BALLON-CIRE D'AUNIS,2021-04-01,2021-04-01 10:43:44.440000+00:00,2021-03-25,...,0.0,0.0,0.0,0.0,P,00,False,None,<NA>,<NA>
0,767775,M14-M14_COM_SUP3500,2020,20003531900090,200035319,303,CA VAR ESTEREL MEDITERRANÉE,2021-04-15,2021-06-16 08:48:00.715000+00:00,NaT,...,NaN,NaN,598080.2,NaN,A,09,True,None,<NA>,<NA>
0,767821,M14-M14_COM_SUP3500,2020,20003531900090,200035319,303,CA VAR ESTEREL MEDITERRANÉE,2021-04-15,2021-06-16 08:48:00.715000+00:00,NaT,...,NaN,NaN,598080.2,NaN,A,09,True,None,<NA>,<NA>
0,801669,M14-M14_COM_500_3500,2021,20004187900012,200041879,800,TERRES DE SAONE,2021-12-14,2021-12-14 09:16:50.103000+00:00,2021-12-13,...,NaN,NaN,NaN,NaN,P,00,False,None,<NA>,<NA>
0,801114,M14-M14_COM_500_3500,2021,20004187900012,200041879,800,TERRES DE SAONE,2021-12-14,2021-12-14 09:16:50.103000+00:00,2021-12-13,...,NaN,NaN,NaN,NaN,P,00,False,None,<NA>,<NA>
0,798035,M4-M49_D,2021,20004217400074,200042174,400,LORIENT AGGLOMERATION,2021-10-11,2021-12-08 10:26:41+00:00,2021-10-11,...,NaN,NaN,NaN,NaN,A,09,False,,<NA>,<NA>
0,798836,M4-M49_D,2021,20004217400074,200042174,400,LORIENT AGGLOMERATION,2021-10-11,2021-12-08 10:26:41+00:00,2021-10-11,...,NaN,NaN,NaN,NaN,A,09,False,,<NA>,<NA>
0,700320,M14-M14_COM_500_3500,2020,20004218200085,200042182,407,CTE COMMUNES DU MONT DES AVALOIRS,2021-03-12,2021-03-12 14:56:48.614000+00:00,NaT,...,NaN,NaN,NaN,NaN,A,00,False,None,<NA>,<NA>
0,665772,M14-M14_COM_500_3500,2020,20004218200085,200042182,407,CTE COMMUNES DU MONT DES AVALOIRS,2021-03-12,2021-03-12 14:56:48.614000+00:00,NaT,...,NaN,NaN,NaN,NaN,A,00,False,None,<NA>,<NA>


Prenons l'exemple de 681458 et 681423

In [50]:
budget_691458 = df_budget[df_budget['Id_Fichier'] == '681458']
budget_681423 = df_budget[df_budget['Id_Fichier'] == '681423']

budget_1er_doublon = df_budget[df_budget['Id_Fichier'].isin(['681458', '681423'])]

print('taille premier :', budget_691458.shape[0])
print('taille seconnd :', budget_681423.shape[0])


taille premier : 103
taille seconnd : 103


Si les lignes sont les mêmes, on passera de 206 lignes à 103. 

On va d'avoir retirer l'Id_Fichier qui est évidemment différent (et qui devrait être la seule diff)

In [57]:
col_autres_que_id = budget_1er_doublon.columns.difference(['Id_Fichier'])

budget_test_unique = budget_1er_doublon.drop_duplicates(subset=col_autres_que_id, keep = 'first')
print(budget_test_unique.shape[0], '\n')
print(budget_test_unique['Id_Fichier'].value_counts())

103 

Id_Fichier
681458    103
Name: count, dtype: int64


Maintenant, il suffit de généraliser ça 

In [88]:
liste_id_doublon_meme = doublon_meme_date['Id_Fichier'].to_list()

df_budget_doublon_meme = df_budget[df_budget['Id_Fichier'].isin(liste_id_doublon_meme)]

print('doublon pure :',df_budget_doublon_meme.shape[0], 'lignes budgets \n')

doublon pure : 28847 lignes budgets 



Certains sont en triples donc on peut pas juste diviser par deux

In [91]:
df_budget_unique_meme_f = df_budget_doublon_meme.drop_duplicates(subset=col_autres_que_id, keep = False)

print('Il y a ', df_budget_unique_meme_f.shape[0], 'lignes budgets unique \
dans les données budgets des documents doublons envoyés au même moment')


Il y a  0 lignes budgets unique dans les données budgets des documents doublons envoyés au même moment


--------------
Maintenant, recherche chez ceux qui ont une diff 

-----------


In [58]:
doublon_diff_date.head(5)

,Id_Fichier,Nomenclature,Exer,Siret,Siren,CodColl,LibelleColl,DteStr,date,DteDec,...,DepFoncN2,RecFoncN2,DepInvN2,RecInvN2,CodTypBud,CodBud,ProjetBudget,Affect,SpecifBudget,FinJur
0,790376,M57-M57,2021,20000480200027,200004802,235,COMMUNAUTE DE COMMUNES PAYS DE FAYENCE,2021-10-29,2021-11-03 07:23:51.768000+00:00,2021-11-02,...,NaN,NaN,NaN,NaN,A,01,True,23501,<NA>,<NA>
0,791034,M57-M57,2021,20000480200027,200004802,235,COMMUNAUTE DE COMMUNES PAYS DE FAYENCE,2021-11-08,2021-11-08 15:01:39.443000+00:00,2021-11-02,...,NaN,NaN,NaN,NaN,A,01,True,23501,<NA>,<NA>
0,804914,M14-M14_COM_SUP3500,2022,20002715900017,200027159,015,VILLE DE DUNKERQUE,2021-12-03,2021-12-22 16:15:22+00:00,2022-12-15,...,0.0,0.0,0.0,0.0,P,00,True,None,<NA>,<NA>
0,813538,M14-M14_COM_SUP3500,2022,20002715900017,200027159,015,VILLE DE DUNKERQUE,2022-02-14,2022-02-14 10:55:28+00:00,2022-12-15,...,0.0,0.0,0.0,0.0,P,00,False,None,<NA>,<NA>
0,818103,M14-M14_COM_500_3500,2022,20003419700018,200034197,284,CTE DE CNES MONTAIGNE MONTRAVEL ET GUR,2022-03-09,2022-03-09 14:48:59.207000+00:00,NaT,...,NaN,NaN,NaN,NaN,A,00,False,None,<NA>,<NA>


Prenons les id_fichier 790376 et 791034

In [59]:
budget_790376 = df_budget[df_budget['Id_Fichier'] == '790376']
budget_791034 = df_budget[df_budget['Id_Fichier'] == '791034']
budget_test_diff = df_budget[df_budget['Id_Fichier'].isin(['791034', '790376'])]

print(budget_790376.shape[0])
print(budget_791034.shape[0])

131
131


In [60]:
budget_dif_unique = budget_test_diff.drop_duplicates(subset=col_autres_que_id, keep = 'first')
print(budget_dif_unique.shape[0], '\n')
print(budget_dif_unique['Id_Fichier'].value_counts())

172 

Id_Fichier
790376    131
791034     41
Name: count, dtype: int64
